<a href="https://colab.research.google.com/github/khodid/2020_SAI_MONING2/blob/master/lab07-2_Using_MNIST_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MNIST Data set
- 학습 내용
  - MNIST Data Set이란 무엇인가?
  - MNIST Classifier Code 


## MNIST Introduction

[Wikipedia: MNIST Database](https://ko.wikipedia.org/wiki/MNIST_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4)

[다운 받는 곳](http://yann.lecun.com/exdb/mnist/)


- 개요:
0-9까지의 숫자 이미지를 손글씨로 쓴 데이터로서, 손으로 쓴 우편번호를 인식하기 위해 마련된 데이터 세트다.

- Train set : train-images-idx3-ubyte.gz / train-labels-idx1-ubyte.gz (6만장의 이미지와 label)
- Test set : T10k-images-idx3-ubyte.gz / T10k-labels-idx1-ubyte.gz (만 장의 이미지와 label)

이 마련되어 있다.

## Example of MNIST

- 28x28 image로 이루어짐.(748 pixel)
- 1 channel of gray image
- 0~9 digits

실제론 이 28X28 이미지를 view라는 함수로 784개로 바꾸어 사용하게 만든다. 

``` python
for X, Y in data_loader:
  X = X.view(-1, 28*28)
```


### Torch Vision 패키지

pytorch에서 불러올 수 있는 유명한 데이터 셋과 mosel architectures과 image transformation들을 담고 있다.

[자세히 알고 싶으면 링크 참조](https://pytorch.org/docs/stable/torchvision/index.html)

In [13]:


# 중략 ... 

mnist_train = dsets.MNIST(root="MNIST_data/", train = True, transform = transforms.ToTensor(), download = True)
# [ Parameter 설명 ]:
# root      - MNIST data가 어느 경로에 있는지 설명 
# train     - True로 할 시 MNIST의 Train set을 불러오고, False로 하면 Test set을 불러옴
# transform - 이미지를 불러올 때 어떤 Transform으로 불러올 것인지. 
#             PyTorch의 경우 이미지는 0에서 1 사이의 값을 [Channel Highth Width] 의 형태로 갖는데, 
#             일반적인 Img 파일의 경우 0-255  사이 값을 [Highth Width Channel] 순서로 값을 갖는다 
#             그래서 이미지의 순서와 값을 Pytorch에 맞게 바꿔 주는 게 ToTensor의 역할
# download  - True로 할 시 Root에 MNIST Data가 존재하지 않으면 다운을 받겠다는 의미

mnist_test = dsets.MNIST(root="MNIST_data/", train = False, transform = transforms.ToTensor(), download = True)
# test set도 마찬가지 함수로 다운 받아 준다.

data_loader = torch.utils.DataLoader(DataLoader = mnist_train, batch_size = 100, shuffle = True, drop_last = True)

# [ Parameter 설명 ]:
# DataLoader - 어떤 데이터를 로드할 것인지
# batch_size - 데이터를 불러올 때 몇 개씩 잘라서 불러올 것인지
# shuffle    - True로 할 시 무작위로 섞어서 불러옴, False일 시 순서대로 불러옴
# drop_last  - batch_size대로 잘라서 가져올 때, 남는 데이터는 어떻게 할지. True로 할 시 나머지 데이터는 사용X


# ... 중략 ... 

for epoch in range(training_epochs):
  # ... 중략...
  for X, Y in data_loader:
      # data는 for문을 이용해서 불러옴.
      # X에는 이미지, Y로는 label을.

      X = X.view(-1, 28*28).to(device)
      # 원래의 X는   [<batch_size>, 1_<channel>, 28_<highth>, 28_<width>] 
      # 바꾼 X는     [<batch_size>, 784]
  

AttributeError: ignored

# Epoch, Batch size, Iteration

머신러닝, 딥러닝, 등등의 분야에서 사용되는 개념이다.

- Epoch: _training set 전체_가 학습에 한 번 사용이 되면 1 epoch이 돌았다고 부른다. 
- batch size: 한 epoch을 돌기 위해서 한꺼번에 테스트 데이터를 다 쓰면 너무 오래 걸림 - 그래서 잘라서 쓸 것. 자르는 묶음의 크기를 batch size라고 부르는 것. 6만장을 100개씩 자르면 600개의 batch를 얻게 될 것이다. 
- Iteration: batch를 몇 번 학습에 사용했느냐. (영어 원문: number of iterations means number of passes, each pass using \[batch size\] number of examples.


예제 : 1000개의 Training Set을 갖고 있고, batch size를 500으로 할 것임. 그렇다면 1 epoch를 수행하기 위해 2 iterations가 필요하다.

### Softmax Classifier를 이용해 실습을 해보겠다...

In [0]:
# [ Data Load를 포함한 사전 세팅용 코드들... ]
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import random

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

# 데이터셋 불러오기

# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [15]:

# [ 여기부터가 강의에 나옴 ]

linear = torch.nn.Linear(784, 10, bias = True).to(device)
# input image는 784개 data, output은 0-9까지의 숫자. bias도 True로 둬서 같이 사용.

training_epochs = 15
batch_size = 100

criterion = torch.nn.CrossEntropyLoss().to(device)   # Softmax에 대해서 자동 계산해주는 함수가 있으므로 class 선언 필요 없음
optimizer = torch.optim.SGD(linear.parameters(), lr = 0.1)

## 위 까지가 기본적인 세팅

# 15회 반복 (전체 데이터 훑기)
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(data_loader)

  # 1 epoch를 돌 때까지 반복
  for X, Y in data_loader:
    X = X.view(-1, 28*28).to(device)      # 아까 설명했듯 이미지 데이터 저장값 재조정하는 과정
    
    optimizer.zero_grad()
    hypothesis = linear(X)
    cost = criterion(hypothesis, Y)
    
    cost.backward()
    optimizer.step()
    avg_cost += cost / total_batch
    

  print("E : ", "%04d" % (epoch+1), "cost = ", "{:.9f}".format(avg_cost))

E :  0001 cost =  0.535468519
E :  0002 cost =  0.359274149
E :  0003 cost =  0.331187546
E :  0004 cost =  0.316578060
E :  0005 cost =  0.307158172
E :  0006 cost =  0.300180733
E :  0007 cost =  0.295130223
E :  0008 cost =  0.290851504
E :  0009 cost =  0.287417054
E :  0010 cost =  0.284379601
E :  0011 cost =  0.281825215
E :  0012 cost =  0.279800683
E :  0013 cost =  0.277808994
E :  0014 cost =  0.276154310
E :  0015 cost =  0.274440825


실제로 몇 만장의 데이터를 돌리니깐 한 epoch 지날 때마다 몇 초씩은 걸리는 걸 체감했다.

### Test

위에 학습한 모델을 테스트 세트를 로드해서 평가해보자.


In [17]:
with torch.no_grad():
  # 이 범위 안에선 gradient를 계산하지 않겠다는 의미의 구문이다.
  # Test를 할 때 이렇게 해둬야 한다고 한다. / 학습할까봐 그런 걸까?

  X_test = mnist_test.test_data.view(-1, 28*28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = linear(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print("Accuracy", accuracy.item())

Accuracy 0.8863000273704529


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


\+ \) matplotlib 라이브러리를 사용한다면 이 예측값과 test 이미지를 직접 시각적으로 확인할 수 있다.